In [1]:
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline 

In [120]:
a = 0.4
b = 0.1
gap = 0.2

P = np.array( [
    [a + gap, b, 0, 0, 0, 0, 0, 0, b],
    [b, a, b, 0, 0, 0, 0, 0, 0],
    [0, b, a, b, 0, 0, 0, 0, 0],
    [0, 0, b, a, b, 0, 0, 0, 0],
    [0, 0, 0, b, a, b, 0, 0, 0],
    [0, 0, 0, 0, b, a, b, 0, 0],
    [0, 0, 0, 0, 0, b, a, b, 0],
    [0, 0, 0, 0, 0, 0, b, a, b],
    [b, 0, 0, 0, 0, 0, 0, b, a] 
]) 

# P = np.maximum(0, P - 1./6) 

Q = np.vstack( [ np.zeros(10), np.hstack( [ (1 - np.sum(P, axis = 1)).reshape(-1,1), P] )])

T = 30000

D = 100

rho = np.max( np.sum(P, axis = 0) ) 


In [121]:
def random_walk( Q, i ):
    
    traj = [i]
    
    counter = 0
    
    next_s = i
    
    while True:
        
        counter += 1
                
        next_s = np.random.multinomial(1, Q[next_s], size=1)
                
        next_s = list(next_s[0]) 
        next_s = next_s.index(1) 
        
        traj.append(next_s) 
        
        if next_s == 0:
            break
            
    return traj

In [122]:
def traj_to_h(trajs, replace = np.inf):
    
    hs = [ ]
    
    for traj in trajs:
        
        h = []
        
        for v in range(1,10): 
            if v in traj:
                h.append( len(traj) - traj.index(v) ) 
            else:
                h.append(replace)
                
        hs.append(h)
    return hs

In [123]:
def get_est( data_matrix, D = 3000 ): 
    
    ns = np.sum( np.array(data_matrix) < np.inf , axis = 0 ) 
    
    hs = [] 
    
    thresholds = np.cumsum( np.array( data_matrix ) < np.inf , axis = 0 ) 
        
    for v in range(9): 

        data_matrix = np.array( data_matrix ) 

        h = np.sum( np.array( data_matrix[:,v]  )[ np.array( data_matrix[:,v] ) \
                                                  < np.sqrt(thresholds[:,v] * D / np.log(t + 2) ) ] )
                            
        hs.append( h / np.maximum( 1, ns[v] ) ) 
        
#     hs = np.sum(hs, axis = 0) / np.maximum( 1, ns ) 
        
    return hs, ns 

# stochastic

In [129]:
import pickle

m = 9; 
rho = np.max( np.sum(P, axis = 0) ) 
Vs = []; 

truncation = False; trajs = [] 
# D = 20
# D = (1 + rho) / (1 - rho)**3

for j in range(10): 
    
    Vs_inner = [] 
    
    hss = []
    nss = []
    
    hs = np.zeros(m); ns = np.zeros(m)

    for t in range(T): 

#         index = hs 
        index = hs + 4 * np.sqrt(1./np.maximum(1,ns)) 

        vt = np.random.choice(np.flatnonzero( index == index.max()) ) + 1
    
        traj = random_walk(Q, vt)   
        
        trajs.append(traj) 
        
        for v in range(1,m+1):  
            
            if v in traj: 
                
                h = len(traj) - traj.index(v) - 1 
                hs[v-1] = (hs[v-1] * ns[v-1] + h ) / (ns[v-1] + 1)
                ns[v-1] += 1 
                    
        Vs_inner.append(vt) 
        
        hss.append(list(hs))
        nss.append(list(ns))
        
    Vs.append(Vs_inner) 
    
    pickle.dump( Vs_inner, open('./raw_data/nodes_UCB_{}'.format(j), 'wb' ) )  
    pickle.dump( hss, open('./raw_data/est_UCB_{}'.format(j), 'wb' ) )  
    pickle.dump( nss, open('./raw_data/ns_UCB_{}'.format(j), 'wb' ) )  

In [125]:
Hs = np.sum( np.linalg.inv( np.identity(9) - P ), axis = 1 ) 
best_H = np.max( Hs ) 

# adversarial

In [30]:
a = 0.3
b = 0.1
gap = 0.1

m = 9; 


P = np.array( [
    [a, b, 0, 0, 0, 0, 0, 0, b],
    [b, a, b, 0, 0, 0, 0, 0, 0],
    [0, b, a, b, 0, 0, 0, 0, 0],
    [0, 0, b, a, b, 0, 0, 0, 0],
    [0, 0, 0, b, a, b, 0, 0, 0],
    [0, 0, 0, 0, b, a, b, 0, 0], 
    [0, 0, 0, 0, 0, b, a, b, 0],
    [0, 0, 0, 0, 0, 0, b, a, b],
    [b, 0, 0, 0, 0, 0, 0, b, a] 
]) 

# P = np.maximum(0, P - 1./6) 

Q = np.vstack( [ np.zeros(10), np.hstack( [ (1 - np.sum(P, axis = 1)).reshape(-1,1), P] )])

rho = np.max( np.sum(P, axis = 0) ) 


In [31]:
Qij = np.linalg.inv( np.identity(9) - P )

In [32]:
l = [[0,1], [2]] 

a = [[1, 2, 3], [4, 5], [6, 7, 8, 9]]
import numpy as np
b = np.zeros([len(a),len(max(a,key = lambda x: len(x)))])
for i,j in enumerate(a):
    b[i][0:len(j)] = j 

# adv. new method

In [110]:
K = 9; T = 20000; 

rho = np.max( np.sum(P, axis = 0) ) 

# np.log( (1 - rho) * K * T * delta ) / np.log(rho) 

eta = 0.001

In [111]:
import pickle 

truncation = False; trajs = [] 
# D = 100 
Vs = [] 

pts = []

for dummy in range(10): 
    
    Vs_inner = [] 
    
    hs = []; hs_raw = []
    hs_true = []; 

    for t in range(T): 
        
#         eta = np.sqrt(1./T) / 10
        
        if t == 0: 
            pt = np.ones(K) / K  
        else: 
            pt = pt * np.exp( eta * np.array(hs[-1]) )  
            pt = pt/ np.sum(pt) 
        
        pts.append(pt)
        
        vt = np.random.choice(np.arange(K), p=pt) + 1
    
        traj = random_walk(Q, vt)   
        
        L = np.random.normal(0.5, 0.1, size = 9) 
        L[0] = L[0] + 0.5 
        
        L = np.minimum( 1, np.maximum(0, L) )
        
        trajs.append(traj) 
        
        Hs = np.zeros(K) 
        
        for v in range(1, m+1):  
            
            if v in traj:
                
                h = (len(traj) - traj.index(v) - 2 + L[traj[-2]-1] ) 
                                
                alpha = 0
                for k in range(K):
                    if k != v - 1:
                        alpha += pt[k] * Qij[k, v-1] 

                H = h / (pt[v-1] + alpha ) 
                
                print(alpha)
                
                hs_raw.append(h) 

                Hs[v-1] = H 
                
        hs.append(Hs) 
                    
        Vs_inner.append(vt) 
        
        hs_true.append( np.dot( np.linalg.inv( np.identity(K) - P ) , 
                               np.sum( P, axis = 0 ) + (1 - np.sum( P, axis = 0 )) * L ) ) 
        
    pickle.dump( Vs_inner, open('./raw_data/nodes_adversarial_{}'.format(dummy), 'wb' ) ) 
        
    pickle.dump( hs_true, open('./raw_data/hs_true_adversarial_{}'.format(dummy), 'wb' ) ) 
        
    Vs.append(Vs_inner) 
    

0.05658754729962159
0.05653734547729922
0.05655825153287836
0.0565965506892373
0.05664040412519845
0.05660844860592726
0.05655802591409363
0.05639392426676398
0.05662506698601651
0.05708664675028292
0.057068202293464806
0.05646454344254081
0.05686078985715081
0.05716963133455752
0.05692064130854198
0.05615948280094621
0.05647901139796683
0.056696375324559324
0.05688365505090299
0.05686653024620956
0.056040868559141414
0.05636227861458076
0.05688425942529897
0.056359324988096746
0.05636429484949729
0.05609055137773375
0.0567920247750376
0.05692335022913448
0.056374094034830784
0.05640312613925754
0.05763172707307051
0.05761312376885541
0.05803337751072144
0.057488382842418126
0.05554526681535413
0.055794754781231415
0.0555254935740721
0.054951874071079446
0.05572494267930078
0.05701038949105139
0.0578000507520003
0.05581276306858672
0.05505418085553643
0.05642066906177809
0.055266967614538705
0.05649147337041226
0.05694749074205718
0.05790587410363565
0.055590926531483745
0.057274786990

0.051373889010547714
0.06397118928896119
0.05151071533232947
0.0596984896373809
0.048476727557117405
0.05959957285886601
0.05957586011244562
0.06646683112353512
0.05973366873094099
0.051349263686770084
0.054334130070168844
0.05418005552076096
0.057495752431895635
0.04802389766058664
0.048109486084778996
0.04808565951603203
0.04806739017930266
0.05919826627453918
0.05454366217810191
0.054526940732636586
0.05909962797706509
0.05179090644826458
0.048347248013371785
0.05778939399387484
0.05794961556369813
0.051664120240068036
0.05894841253878594
0.06639952513002317
0.0665425436996689
0.05899271594252168
0.048566003856103865
0.05777373343301632
0.048152154921672456
0.04852392913829889
0.05124854117206915
0.0591567052732272
0.05866388390189042
0.04879681740540265
0.06351152421298539
0.05914469037512686
0.06382393609024022
0.05915805827427373
0.0576367672796538
0.0638692971102875
0.06383789651188512
0.05984764978005629
0.05410183281277122
0.052336174332784834
0.06378644351817507
0.04801214674

0.0702616894105014
0.056874090843970194
0.05664827271577559
0.07402193201608727
0.05198544702909174
0.05056992100811767
0.050553358796225786
0.05328341048136104
0.04412104195129707
0.051302317022581166
0.07116422842122778
0.05038867863149647
0.0558800783703276
0.05104052845035321
0.054067713805099504
0.0540731299901544
0.05589280844881077
0.07120406239923177
0.07118094324500088
0.051234516649077744
0.04500123920820608
0.07385622302918056
0.07118611161799075
0.05150705088668217
0.053869374194504875
0.05128690616783525
0.07125054703176241
0.0557336311685504
0.07359768367708686
0.05199909112245359
0.04998274437507595
0.055677265228617316
0.05168386027895588
0.05164283233167273
0.05160198612964928
0.07181780810080647
0.05552460841796038
0.051699217633189326
0.07471126480207695
0.05312879379234448
0.0443971127961675
0.05152729985245494
0.051487377953372854
0.07242984090189393
0.05169862470728536
0.05165960544998187
0.0516199590927211
0.052887064605748
0.04462648577734752
0.05503466803122488

0.051154558309902995
0.08806222881127757
0.04645015100395841
0.053986785506773174
0.04642356217824498
0.051286040492751865
0.04138563853358923
0.046499297712891574
0.04832930276146506
0.05107896150089112
0.08769904304202067
0.04166972228648453
0.04662447298039431
0.04177158802675936
0.046875421875594035
0.05075775559934252
0.04935329157537821
0.051201541564329406
0.08133695420264536
0.042239382179405364
0.05067610173223891
0.08681899475433673
0.04702324505685097
0.050594592330050536
0.05047084083078379
0.08203489212697147
0.053147389658791164
0.041925767339563484
0.04703628896613077
0.05328800700712919
0.0873018123348533
0.0421776745738546
0.05105137254550923
0.049247893591470346
0.05048384688754673
0.04912519206761667
0.04910894932439481
0.05121579692354872
0.08156365431960041
0.05048138243148246
0.08166017219312903
0.08163216385483035
0.05332875260199827
0.04654803069669398
0.050565808766041845
0.08749419415554605
0.051474168910560926
0.04177689611072897
0.04745032741572527
0.0505063

0.04722740082403001
0.047208039446268266
0.09600153681284257
0.05639540538211748
0.04126208469207387
0.04738888516288071
0.08642391563061118
0.04389472086112979
0.08650387254017472
0.047675782689337455
0.04759695850651642
0.08669924557034359
0.05678535537307563
0.057008632121664383
0.04794806762382471
0.08739996000082566
0.04633611282595046
0.08721532751465158
0.09558447302980136
0.04847912840040867
0.04835333554759941
0.09588374167016628
0.04299018712783832
0.0573272352355389
0.05731486320431885
0.05730819742453619
0.041927133585430626
0.04155655987519905
0.041646235328752716
0.043226348413528466
0.04719977910697117
0.0463096542927465
0.08664220647241579
0.04773776430227047
0.04625637122572672
0.09504849650585087
0.047667949157817885
0.04430922844681235
0.04424048487164442
0.04755571110020914
0.047390776844513385
0.04264670733712552
0.04731261108577288
0.08707760590637409
0.046134243566257435
0.0425417808390768
0.04250272510305741
0.043928978695032435
0.04716539827741262
0.09529143607

...

4.140512386578413e-05
4.136432037508498e-05
4.132297828725647e-05
4.12816775177279e-05
4.124143523204434e-05
4.1202503060672827e-05
0.21599141805362077
4.1094748773144775e-05
0.21599589814039363
4.1003359062728925e-05
4.09623777287243e-05
4.088360720178508e-05
4.084387691117337e-05
4.080402887347097e-05
4.0763246755123966e-05
0.21581593675133112
4.0644286536772505e-05
4.061058057700194e-05
4.040803876565698e-05
4.03315506825611e-05
0.21583476732329346
4.025546013959316e-05
4.021763197518477e-05
4.017743591741457e-05
4.013728003265831e-05
4.0057088312366046e-05
4.002133682693941e-05
3.9981336951812523e-05
3.9905237105861697e-05
3.9865353263947824e-05
3.9825555089490214e-05
3.9787019547706976e-05
3.970752760265166e-05
0.21586144520081846
3.961533966799801e-05
3.957967546766731e-05
0.21605365882802985
3.952274220579911e-05
3.9484380933658535e-05
0.21587115410047583
3.943614111534792e-05
3.935809759376514e-05
0.2160620878258263
3.927070147654152e-05
3.9231451807309614e-05
0.215882259402835

2.963224837576222e-05
2.960263173248339e-05
2.9545733868324744e-05
0.2162179842795947
2.950734071731929e-05
2.9477848912578473e-05
2.9452268192054748e-05
2.9422831429367622e-05
2.939342408715813e-05
0.21638113665524575
2.9352377888225512e-05
2.9323231079895795e-05
2.926593345241857e-05
0.21623029158249957
0.03161584349973369
2.919054630724421e-05
2.910310768806956e-05
2.907401989290325e-05
2.901593131953887e-05
2.8988517073089465e-05
0.2163974945949591
2.8897947765025994e-05
2.8814007023510665e-05
2.8786675521068403e-05
2.873275197611403e-05
0.2164074661874121
2.860360485772673e-05
0.21626095536210602
2.8532396638766008e-05
2.850692230407751e-05
2.8484853718009403e-05
2.8460172550150612e-05
0.21626740836833866
0.03161933416138641
2.840817213708955e-05
2.8383869567641738e-05
2.832887414075203e-05
2.8303550873814057e-05
2.8275879273633198e-05
2.822080519260315e-05
2.8196191493434322e-05
2.8172968827731677e-05
0.032669120226215494
0.21642990504850668
2.811073853282948e-05
2.80265343533411

3.1862775324884814e-05
3.183092938313554e-05
3.180386256029471e-05
3.174031998028142e-05
3.1676904352099e-05
3.1646650684327045e-05
0.21596504994357363
3.1601480670796864e-05
3.154063085634058e-05
0.21616383946938827
3.146513936685539e-05
3.143369083973956e-05
3.1402273743657565e-05
3.1373379351007766e-05
3.1342022532871205e-05
0.21597996824241394
3.130033039368937e-05
3.1144221519085124e-05
3.1081996837759394e-05
3.1054392990296675e-05
3.1023354981226133e-05
3.09935307706049e-05
3.086980640146088e-05
0.21619225894740582
3.0703380127981766e-05
3.067613492969263e-05
3.064547496869477e-05
3.061484565064703e-05
0.21601540426920152
3.05715846885081e-05
3.054109571065512e-05
3.051057071397291e-05
0.21620770688572755
3.046650891400882e-05
0.2162095925650778
3.0333385677662417e-05
0.21602930557410063
3.0265576217669776e-05
0.21603287448893435
3.02206386482469e-05
3.0193878643246367e-05
0.2160364491862062
3.0147217057225954e-05
0.2162236756373676
3.004368388936789e-05
0.21604393710859626
2.997

1.8481719493033496e-05
1.8463247317473087e-05
1.8444793604242338e-05
1.837116257426558e-05
1.8280077616085926e-05
1.826243184190051e-05
0.21665735887889778
1.823395455589966e-05
0.21676136642273666
1.8172984700647422e-05
0.21666158326538607
0.21676386621093605
1.805102300488635e-05
1.8032981296441477e-05
0.21666866950312394
0.21677007982148352
1.789228658520353e-05
1.7875110491138392e-05
0.2166764036598007
0.03166090296155728
0.004651096215259144
1.9177557286972743e-05
1.9158389645046356e-05
0.2166275084969527
1.911300097456489e-05
0.21662960578784776
1.9027978212754085e-05
1.900944959694961e-05
0.21672385920267584
1.8979544819899172e-05
1.896143305571688e-05
1.892354865502263e-05
1.8885739945140204e-05
1.8868667461293277e-05
1.8849808544263e-05
1.8812147162812354e-05
0.21664366085444675
1.878424613384553e-05
1.874671574228492e-05
1.8728175832698405e-05
0.2167353093233746
1.8655391458923874e-05
1.8636745704109694e-05
0.21673908261643018
1.8609917645085584e-05
1.8554172262638055e-05
1.8

3.5460412205675385e-05
0.2158283903409313
3.52744455059703e-05
0.21583735221802353
3.522655649083274e-05
3.5191348655403325e-05
3.5156176007977744e-05
3.508593586811004e-05
0.21584622224796957
0.03308203506829129
0.21606817181445465
3.484158928266574e-05
3.470250404810638e-05
3.466781997021149e-05
3.463394018672954e-05
3.460127554017695e-05
3.4532144031163834e-05
3.4505893786786454e-05
0.033059273058379295
0.2160922027485283
3.436032687392561e-05
3.432598478089474e-05
3.4291677010667044e-05
3.4224963364090814e-05
3.419075655854894e-05
0.21610507554494648
3.3987686680977906e-05
3.395663413245803e-05
3.392775706113812e-05
3.386480749769942e-05
0.2161183574190862
3.381174621136395e-05
3.377795239112223e-05
0.0330311627372286
0.21612189225931966
3.3687934800185865e-05
3.365426472194911e-05
3.3620628294919374e-05
3.358786585074627e-05
0.03302380717041572
0.21612966071645295
3.347237745175033e-05
0.2161343906291147
3.335960997106681e-05
3.3326268033737294e-05
3.32596838546309e-05
3.322657740

In [112]:
alpha

2.8035412324119714e-05

# Adv. standard

In [113]:
K = 9; T = 20000; 

rho = np.max( np.sum(P, axis = 0) ) 
# np.log( (1 - rho) * K * T * delta ) / np.log(rho) 

eta = 0.001

In [114]:
import pickle 

truncation = False; trajs = [] 
# D = 100 
Vs = [] 

# hs_true = []; 

pts = []

for dummy in range(10): 
    
    Vs_inner = [] 
    
    hs = []; hs_raw = []
    
    hs_true = []; 

    for t in range(T): 
        
#         eta = np.sqrt(1./T) / 10
        
        if t == 0: 
            pt = np.ones(K) / K  
        else: 
            pt = pt * np.exp( eta * np.array(hs[-1]) )  
            pt = pt / np.sum( pt ) 
        
        pts.append(pt)
        
        vt = np.random.choice(np.arange(K), p=pt) + 1
    
        traj = random_walk(Q, vt)   
        
        L = np.random.normal(0.5, 0.1, size = 9) 
        L[0] = L[0] + 0.5
        
        L = np.minimum( 1, np.maximum(0, L) )
        
        trajs.append(traj) 
        
        Hs = np.zeros(K) 
        
        h = (len(traj) - traj.index(vt) - 2 + L[traj[-2]-1] ) 

        H =  h / (pt[vt-1] + 0.001) # to avoid numerical instability  

        Hs[vt-1] = H 
                
        hs.append(Hs) 
                    
        Vs_inner.append(vt) 
        
        hs_true.append( np.dot( np.linalg.inv( np.identity(K) - P ) , 
                               np.sum( P, axis = 0 ) + (1 - np.sum( P, axis = 0 )) * L ) ) 
        
    pickle.dump( Vs_inner, open('./raw_data/nodes_adversarial_standard_{}'.format(dummy), 'wb' ) ) 
        
    pickle.dump( hs_true, open('./raw_data/hs_true_adversarial_standard_{}'.format(dummy), 'wb' ) ) 
        
    Vs.append(Vs_inner) 
    